In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: opencv-python in c:\users\nawaphon\appdata\local\programs\python\python312\lib\site-packages (4.10.0.84)




[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Get Data

In [31]:
import cv2
import mediapipe as mp
import os

# สร้างตัวแปร mediapipe สำหรับการตรวจจับมือ
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# เปิดกล้องหรือวิดีโอ
cap = cv2.VideoCapture(0)

# สร้างโฟลเดอร์เพื่อเก็บรูปภาพ
output_folder = 'C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# กำหนดจำนวนภาพสูงสุดที่ต้องการบันทึก
max_images = 1200

with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    frame_count = 0
    while cap.isOpened() and frame_count < max_images:
        ret, frame = cap.read()
        if not ret:
            break

        # เปลี่ยนเป็นสี RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # ตรวจจับมือในภาพ
        results = hands.process(image)

        # เปลี่ยนกลับเป็น BGR เพื่อแสดงผล
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # วาดจุด landmarks ถ้าตรวจพบ
        if results.multi_hand_landmarks:
            all_x_min, all_y_min, all_x_max, all_y_max = [], [], [], []
            for hand_landmarks in results.multi_hand_landmarks:
                # วาด landmarks บนมือ
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # คำนวณตำแหน่งของกรอบมือ
                h, w, _ = image.shape
                x_min = int(min([landmark.x * w for landmark in hand_landmarks.landmark]))
                y_min = int(min([landmark.y * h for landmark in hand_landmarks.landmark]))
                x_max = int(max([landmark.x * w for landmark in hand_landmarks.landmark]))
                y_max = int(max([landmark.y * h for landmark in hand_landmarks.landmark]))

                all_x_min.append(x_min)
                all_y_min.append(y_min)
                all_x_max.append(x_max)
                all_y_max.append(y_max)
                
                x_min = max(0, min(all_x_min) - 20)
                y_min = max(0, min(all_y_min) - 20)
                x_max = min(w, max(all_x_max) + 20)
                y_max = min(h, max(all_y_max) + 20)

                # ตัดภาพ ROI (Region of Interest) สำหรับมือ
                hand_roi = image[y_min:y_max, x_min:x_max]

                # แสดงภาพ ROI ที่ตัดจากมือ
                if hand_roi.size > 0:
                    cv2.imshow('Hand ROI', hand_roi)

                    # บันทึกภาพ ROI เป็นไฟล์ .png ในโฟลเดอร์
                    image_filename = os.path.join(output_folder, f'frame_{frame_count}.jpg')
                    cv2.imwrite(image_filename, hand_roi)
                    print(f"Saved: {image_filename}")
                    frame_count += 1

                    # ถ้าบันทึกภาพครบ 1400 ภาพแล้วให้หยุด
                    if frame_count >= max_images:
                        print("Reached maximum image limit.")
                        break

        # แสดงภาพที่ตรวจจับ
        cv2.imshow('Hand Detection', image)

        if cv2.waitKey(5) & 0xFF == 27:  # กด ESC เพื่อหยุด
            break

cap.release()
cv2.destroyAllWindows()

print(f"Images saved to {output_folder}")


Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_0.jpg
Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_1.jpg
Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_2.jpg
Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_3.jpg
Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_4.jpg
Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_5.jpg
Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_6.jpg
Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_7.jpg
Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_8.jpg
Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_9.jpg
Saved: C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai/20\frame_10.jpg
Saved: C:/Users/nawa

## Train Data

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path ไปยัง dataset ที่มีหลายโฟลเดอร์ย่อยสำหรับแต่ละคลาส
data_dir = 'C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/Thai'

# สร้าง ImageDataGenerator สำหรับการโหลดข้อมูลจากโฟลเดอร์ และการแบ่งข้อมูล
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.3)  # แบ่ง 70% สำหรับเทรน, 30% สำหรับ validation

# โหลดข้อมูลการเทรนจากโฟลเดอร์ (ที่มีหลายคลาส) และกำหนด subset='training'
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(148, 148),  # ขนาดภาพที่โมเดลคาดหวัง
    batch_size=32,           # จำนวนภาพในแต่ละ batch
    class_mode='categorical', # ป้ายกำกับเป็นแบบ one-hot encoding
    subset='training'         # ใช้ข้อมูลส่วนนี้สำหรับเทรน
)

# โหลดข้อมูล validation จากโฟลเดอร์ โดยกำหนด subset='validation'
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(148, 148),
    batch_size=32,
    class_mode='categorical',
    subset='validation'       # ใช้ข้อมูลส่วนนี้สำหรับ validation
)

# สร้างโมเดล CNN สำหรับการจำแนก
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(148, 148, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')  # จำนวนคลาสที่ตรวจจับได้
])

# คอมไพล์โมเดล
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# เทรนโมเดล
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10  # จำนวน รอบ
)

# บันทึกโมเดล
model.save('C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/AI_python.h5')

# แสดงลำดับของคลาส
class_indices = train_generator.class_indices
print("Class Indices:", class_indices)


Found 16800 images belonging to 20 classes.
Found 7200 images belonging to 20 classes.


c:\Users\nawaphon\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\nawaphon\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 240s 445ms/step - accuracy: 0.8357 - loss: 0.7061 - val_accuracy: 0.9289 - val_loss: 0.3938
Epoch 2/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 75us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


c:\Users\nawaphon\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


525/525 ━━━━━━━━━━━━━━━━━━━━ 111s 212ms/step - accuracy: 0.9991 - loss: 0.0040 - val_accuracy: 0.9258 - val_loss: 0.4028
Epoch 4/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 21us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 108s 206ms/step - accuracy: 1.0000 - loss: 5.4289e-04 - val_accuracy: 0.9274 - val_loss: 0.5456
Epoch 6/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 21us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 102s 194ms/step - accuracy: 0.9974 - loss: 0.0129 - val_accuracy: 0.9176 - val_loss: 0.5863
Epoch 8/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 19us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 105s 200ms/step - accuracy: 0.9993 - loss: 0.0024 - val_accuracy: 0.9054 - val_loss: 0.9732
Epoch 10/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 20us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


Class Indices: {'1': 0, '10': 1, '11': 2, '12': 3, '13': 4, '14': 5, '15': 6, '16': 7, '17': 8, '18': 9, '19': 10, '2': 11, '20': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19}


## Test Model

In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
from PIL import ImageFont, ImageDraw, Image

# สร้างตัวแปร mediapipe สำหรับการตรวจจับมือ
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


# โหลดโมเดลที่ผ่านการเทรนแล้ว
model = load_model("C:/Users/nawaphon/Documents/65050454/Hard Project/Detect_Hand/AI_python.h5")

# กำหนดคลาสตัวอักษรที่โมเดลจะทำนาย
classes = ['พ่อ', 'ดีใจ', 'มีความสุข', 'ชอบ', 'ไม่สบาย', 'เข้าใจแล้ว', 'เศร้า', 'ยิ้ม', 'โชคดี', 'หิว', 'ชอบ',
           'แม่', 'ขอความช่วยเหลือ', 'ฉัน', 'เขา', 'ขอโทษ', 'ขอโทษ', 'เป็นห่วง', 'เป็นห่วง', 'รัก', 'เขา',
           'สวัสดี', 'แม่', 'สวัสดี', 'เสียใจ', 'เสียใจ', 'ขอบคุณ', 'ยิ้ม', 'อิ่ม', 'แม่', 'รัก',
           'รัก', 'เข้าใจแล้ว', 'เข้าใจแล้ว', 'ขอความช่วยเหลือ', 'ห', 'ฬ', 'อ', 'ฮ']
# เปิดกล้องหรือวิดีโอ
cap = cv2.VideoCapture(0)

# โหลดฟอนต์ภาษาไทย
fontpath = "C:/Users/nawaphon/Documents/65050454/Hard Project/myproject/Bethai.ttf"  # ใส่ที่อยู่ไฟล์ฟอนต์ภาษาไทยที่ดาวน์โหลด
font = ImageFont.truetype(fontpath, 30)

with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (640, 480))

        # เปลี่ยนเป็นสี RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # ตรวจจับมือในภาพ
        results = hands.process(image)

        # เปลี่ยนกลับเป็น BGR เพื่อแสดงผล
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # วาดจุด landmarks ถ้าตรวจพบ
        if results.multi_hand_landmarks:
            all_x_min, all_y_min, all_x_max, all_y_max = [], [], [], []
            for hand_landmarks in results.multi_hand_landmarks:
                # วาด landmarks บนมือ
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # คำนวณตำแหน่งของกรอบมือ
                h, w, _ = image.shape
                x_min = int(min([landmark.x * w for landmark in hand_landmarks.landmark]))
                y_min = int(min([landmark.y * h for landmark in hand_landmarks.landmark]))
                x_max = int(max([landmark.x * w for landmark in hand_landmarks.landmark]))
                y_max = int(max([landmark.y * h for landmark in hand_landmarks.landmark]))

                # เก็บค่าของแต่ละมือไว้
                all_x_min.append(x_min)
                all_y_min.append(y_min)
                all_x_max.append(x_max)
                all_y_max.append(y_max)

            # คำนวณกรอบรอบทั้งสองมือ
            x_min = max(0, min(all_x_min) - 20)
            y_min = max(0, min(all_y_min) - 20)
            x_max = min(w, max(all_x_max) + 20)
            y_max = min(h, max(all_y_max) + 20)

            # วาดกรอบครอบทั้งสองมือ
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (102, 0, 0), 2)

            # ตรวจสอบว่ามีการตัดภาพและมีขนาดที่เหมาะสม
            hand_roi = image[y_min:y_max, x_min:x_max]

            if hand_roi.size > 0:
                # ปรับขนาดให้ตรงกับที่โมเดลคาดหวัง (148x148)
                hand_roi_resized = cv2.resize(hand_roi, (148, 148))

                # ทำให้ภาพเป็น array และปรับขนาดให้เหมาะกับการป้อนเข้าโมเดล
                hand_roi_resized = np.array(hand_roi_resized, dtype="float32") / 255.0
                hand_roi_resized = np.expand_dims(hand_roi_resized, axis=0)

                # ทำนายตัวอักษรจากภาพที่ตัด
                try:
                    predictions = model.predict(hand_roi_resized)
                    predicted_class = np.argmax(predictions[0])
                    predicted_letter = classes[predicted_class]
                    confidence = predictions[0][predicted_class] * 100  # เปอร์เซ็นต์ความมั่นใจ

                    # แปลงภาพจาก OpenCV เป็นรูปแบบที่ PIL ใช้
                    image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                    draw = ImageDraw.Draw(image_pil)

                    # ข้อความที่จะแสดง
                    text_th = f'ทำนาย: {predicted_letter} {confidence:.1f}%'

                    # หาขนาดข้อความ
                    bbox = draw.textbbox((0, 0), text_th, font=font)
                    text_width = bbox[2] - bbox[0]
                    text_height = bbox[3] - bbox[1]

                    # วาดกรอบและข้อความ
                    draw.rectangle([x_min, y_min - text_height - 10, x_min + text_width, y_min], fill=(0, 0, 102))
                    draw.text((x_min + 5, y_min - text_height - 5), text_th, font=font, fill=(255, 255, 255))

                    # แปลงกลับเป็นภาพแบบ OpenCV เพื่อแสดงผล
                    image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

                except Exception as e:
                    print(f"Error in prediction: {e}")

        # แสดงภาพที่ตรวจจับ
        cv2.imshow('Hand Detection', image)

        if cv2.waitKey(5) & 0xFF == 27:  # กด ESC เพื่อหยุด
            break

cap.release()
cv2.destroyAllWindows()

c:\Users\nawaphon\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━